# Implementation and optimization of the mean-timer technique in drift tube detectors

### Description

The data to be analyzed have been gathered by a series of (4) particle detectors. Such detectors were composed by four layers of cells, each measuring the time of the electronic signal released at the passage of a charged particle. More in detail, the particle ionizes the gas within the cell, and the produced electrons drift at constant velocity towards the anodic wire at the center of cell. 
The goal is to precisely assess the position where a particle has traversed a given cell; combining the information of four consecutive cells it is then possible to deduce the trajectory of the particle.

Let's use the following picture as reference:

![alt text](./meantimer.png)


The charged particle is represented by the red arrow. The four layers of cells are visible, only two cells each were displayed whereas the actual detector has 16 cells per layer. The black dots represent the anodic wires towards which the ionization produced by the charged particle drifts. Each cell measures the time of the ionization signal with rispect to a given counter (see later).
There is a left-right ambiguity on where the particle has passed within a cell (left hypothesis represented by green X, right hypothesis represented by the blue X).

The numbering of the cells is somewhat uncommon, and follows the notation displayed in the picture.

The actual time taken by the ionization to reach the anodic wire (drift time) is proportional to the distance:

$$
x = v_d (t - t_0)
$$

where $v_d$ is the constant drift velocity, $t$ is the time recorded by the electronics and $t_0$ is a time pedestal - which needs to be determined - corresponding to the time where the particle crosses the cell (the drift time is $t-t_0$). The determination of $t_0$ is possible thanks to the geometry of the layers, and specifically by the fact that they are staggered by exactly half a cell. This allow to exploit a (set of) mean-timer equation(s) to spot (trigger) the passage of a particle, e.g., thanks to Talete's theorem, the following relation holds:

$$
T_{Max} = \frac{t_1+t_3}{2}+t_2
$$

where $t_1$, $t_2$, $t_3$ are the drift times of those three cells and $T_{Max}$ is the maximum drift time, which is a known quantity: $T_{Max} = \frac{L}{2 v_d} = 390$ ns ($L=42$ mm is the lenght of the cell). When hits are found in a column of cells like the one displayed in the picture, the equation above can be used to determine the time pedestal ($t_0$), thus the drift times in each cell, and thus the position of the hit within the cell (although with the left-right ambiguity).

As it can be guessed from the picture, the same equation would hold for the drift times $t_2$, $t_3$, $t_4$ and similar ones for the other combinations (patterns).

A complete description of the mean-timer technique can be found in the attached document (NOTE2007_034.pdf), which includes all the possible cell and left/right patterns (pages 9, 10, 11).


### Datasets

#### Data Format

The raw data (hexadecimal format) have been preprocessed to produce csv files, each row being formatted in the same way, with 6 fields encoding the address of the read out channel (i.e. the cell) and the actual time measurement:

* *HEAD*: always 1 (useless)
* *FPGA*: 0 or 1
* *TDC_CHANNEL*: in the range 1-128, special values are present too (137, 138, 139) 
* *ORB_CNT*: in the range $0-2^{32}$
* *BX*: in the range $0-3564$
* *TDC_MEAS*: in the range $1-30$

where the channel are mapped to the four detectors in the following ways:

* Detector 1 $\rightarrow$ FPGA 0, TDC_CHANNEL in [1-64]
* Detector 2 $\rightarrow$ FPGA 0, TDC_CHANNEL in [65-128]
* Detector 3 $\rightarrow$ FPGA 1, TDC_CHANNEL in [1-64]
* Detector 4 $\rightarrow$ FPGA 1, TDC_CHANNEL in [65-128]

the time measurement is code similary to standard time (h:m:s), the time in nanoseconds is given by:

$$
t=ORB\_CNT*3564*25+BX*25+TDC\_MEAS*25/30
$$

As it will explained later, actually the orbit information will not be necessary, i.e. the relevant time information will be the one from BX and TDC_MEAS.

Every row in the dataset represents a hit in the detector (a green/blue dot in the picture above).

#### Trigger and trigger-less data aquisition

The data have been taken exploiting an acquisition system which read the detectors sensors at 40 MHz (every 25 ns), without the need for an external trigger. Imagine this as a video recording versus a camera taking pictures when ordered to (external trigger). To limit the bandwidth a "zero-suppression" is applied, i.e. sensors' data is recorded only when they contain something (i.e. when an hit is present in a cell). 

Even though not used in the data acquisition, two trigger systems were operative. Their output consist in a special 64 bit word (a row in the .csv file) enconding the trigger information, i.e. the kind of trigger fired, and the timing at which the particle passed:

* two scintillator detectors, placed parallel to the plane of the cells, when TDC_CHANNEL=137 or 138
* an electronic trigger which recognizes hits in algigned cells, when TDC_CHANNEL=139

#### Data location and runs

In your assigned VM (LCPmodA-Y4-Gr1, IP: 10.67.22.126) you find the relevant datasets on `\data`. Datasets have been grouped into "runs", where 

* 260-262 are "calibration" runs
* 333 is a "physics" run

The difference between the two is that in the calibration run, most of the hits can be associated to a particle's track, whereas in the physics run most of the hits are due to background events.


### Assignments

An "event" can be thought as the passage of a particle through the detector(s), i.e. a collection of hits produced by the particle's track. The datasets are not composed of events, instead they consist of just a series of hits.
The goal of the project is to process the data to reformat them as list of events.

Note that the rate of passage of particles through the detector was rather limited, i.e. the chance to get two particle within the same orbit is extremely small.

The following assignments should be accomplished using Pandas and all its functionalities. The datasets from the calibration runs should be looked at first for simplicity.

- Part 0
    * Group the hits (rows) by orbits, identify and create events, excluding the external trigger information (for the moment). An event must be a collection of hits expressed in terms of coordinates (chamber, layer, cell) and position within the cell (with the left-right ambiguity NOT resolved). 
    * Verify graphically that the hits in the events align in a pattern similar to the one displayed in the picture above (just few events, for validation purposes).

- Part 1
    * Implement the mean-timer technique to determine the time pedestal ($t_0$), the crossing angle ($\alpha$), and solve the left/right ambiguity. The mean-timer should be applied on each detector (4 layers) and on an event-by-event basis. Consider only layers and orbits with 3 or 4 hits that are roughly aligned. For the sake of simplicity, neglect events that have more than 4 hits or events with detector noise. Refer to the documentation of the mean-timer techniques to consider all the appropriate patterns.
    
- Part 2
    * In some cases, more than one pattern is applicable to the same event (for instance, when 4 hits out of 4 are present), yielding different time pedestals. Find a way to select the most appropriate one.
    
- Part 3 
    * Starting from a minimal working example of the previous items (e.g. a loop on the events), try to optimize the running time, and measure the improvement for each step. Possible ways to achieve this are: reduction of the possible combinations, vectorization of some operations, using Numba or CUDA to accelerate blocks of code that are computationally demanding.
    

### Contacts

* Marco Zanetti <marco.zanetti@unipd.it>
* Alberto Zucchetta <alberto.zucchetta@pd.infn.it>
* Jacopo Pazzini <jacopo.pazzini@unipd.it>